# Add Search Tool as Node in the graph

https://app.tavily.com/home



# 1. Install the search engine

In [ ]:
pip install -U langchain-tavily --break-system-packages

# 2. Configure your environment
Configure your environment with your search engine API key:

In [ ]:
import os

# os.environ["TAVILY_API_KEY"] = "......."

from dotenv import load_dotenv
# from langchain import OpenAI, SQLDatabase, SQLDatabaseChain

load_dotenv()

# Access environment variables
tavily_key = os.getenv('TAVILY_API_KEY')


print(f"API Key: {tavily_key}")


# 3. Define the tool¶
Define the web search tool:

In [ ]:
from langchain_tavily import TavilySearch

tool = TavilySearch(max_results=2)
tools = [tool]
# tool.invoke("What's a 'node' in LangGraph?")

# 4. Define the graph¶
For the StateGraph you created in the first tutorial, add bind_tools on the LLM. This lets the LLM know the correct JSON format to use if it wants to use the search engine.

Let's first select our LLM:

In [ ]:
pip install -U "langchain[openai]" --break-system-packages

In [ ]:
# import os
from langchain.chat_models import init_chat_model

# os.environ["OPENAI_API_KEY"] = "......"
# openapi_key = os.getenv('OPENAI_API_KEY')
# print(f"OpenAI API Key: {openapi_key}")

llm = init_chat_model("openai:gpt-4.1-mini")
# llm = init_chat_model("ollama:llama3.2")

### We can now incorporate it into a StateGraph:

In [ ]:
from typing import Annotated

from langchain_tavily import TavilySearch
from typing_extensions import TypedDict

from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition

class State(TypedDict):
    messages: Annotated[list, add_messages]

graph_builder = StateGraph(State)

tool = TavilySearch(max_results=2)
tools = [tool]
llm_with_tools = llm.bind_tools(tools)

def chatbot(state: State):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}

graph_builder.add_node("chatbot", chatbot)

tool_node = ToolNode(tools=[tool])
graph_builder.add_node("tools", tool_node)

graph_builder.add_conditional_edges(
    "chatbot",
    tools_condition,
)
# Any time a tool is called, we return to the chatbot to decide the next step
graph_builder.add_edge("tools", "chatbot")
graph_builder.add_edge(START, "chatbot")
graph = graph_builder.compile()

# 7. Visualize the graph (optional)

In [ ]:
from IPython.display import Image, display

try:
    display(Image(graph.get_graph().draw_mermaid_png()))
except Exception:
    # This requires some extra dependencies and is optional
    pass

# 8. Ask the bot questions
Now you can ask the chatbot questions outside its training data:

In [ ]:
def stream_graph_updates(user_input: str):
    for event in graph.stream({"messages": [{"role": "user", "content": user_input}]}):
        for value in event.values():
            print("Assistant:", value["messages"][-1].content)

while True:
    try:
        user_input = input("User: ")
        if user_input.lower() in ["quit", "exit", "q"]:
            print("Goodbye!")
            break

        stream_graph_updates(user_input)
    except:
        # fallback if input() is not available
        user_input = "What do you know about LangGraph?"
        print("User: " + user_input)
        stream_graph_updates(user_input)
        break